In [3]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score

# === Load Processed Data ===
df = pd.read_csv("D:/flipkart_recommendation/data/final_processed_reviews.csv")

# === Step 1: Simulate Rating from Sentiment Polarity ===
df["simulated_rating"] = df["sentiment_score"] * 5

# === Step 2: RMSE for Recommendations ===
def calculate_rmse(df):
    actual = df.groupby("product_id")["simulated_rating"].mean()
    predicted = np.random.uniform(2.5, 4.5, len(actual))  # simulated predictions
    return np.sqrt(mean_squared_error(actual, predicted))

# === Step 3: Precision@K ===
def precision_at_k(true_list, predicted_list, k=5):
    true_set = set(true_list)
    pred_set = set(predicted_list[:k])
    return len(true_set & pred_set) / k

# === Step 4: F1 & Accuracy for Sentiment (using manually labeled test set) ===
def evaluate_sentiment_classification():
    test_df = pd.read_csv("D:/flipkart_recommendation/data/sentiment_test_labeled.csv")
    
    test_df["predicted_sentiment"] = test_df["review_text"].apply(lambda x: (
        "positive" if TextBlob(x).sentiment.polarity > 0.1
        else "negative" if TextBlob(x).sentiment.polarity < -0.1
        else "neutral"
    ))
    
    acc = accuracy_score(test_df["true_sentiment"], test_df["predicted_sentiment"])
    f1 = f1_score(test_df["true_sentiment"], test_df["predicted_sentiment"], average='macro')
    
    return acc, f1

# === Run Evaluation ===
rmse = calculate_rmse(df)
acc, f1 = evaluate_sentiment_classification()
p_at_3 = precision_at_k(["pixel 7a", "moto edge", "samsung a54"], ["pixel 7a", "vivo y56", "moto edge"], 3)

# === Print Results ===
print("📊 Evaluation Metrics")
print(f"✅ RMSE (Simulated Ratings): {rmse:.3f}")
print(f"✅ Precision@3: {p_at_3:.2f}")
print(f"✅ Sentiment Accuracy: {acc:.2f}")
print(f"✅ Sentiment F1 Score: {f1:.2f}")


📊 Evaluation Metrics
✅ RMSE (Simulated Ratings): 1.449
✅ Precision@3: 0.67
✅ Sentiment Accuracy: 1.00
✅ Sentiment F1 Score: 1.00
